In [7]:
# All imports
import os, sys
import datetime
import time, shutil
import librosa
import librosa.display
import pysrt
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import subprocess

from utils import normalize, loss_cal, optim
from tensorflow.contrib import rnn
from sklearn.metrics.pairwise import cosine_similarity
from scipy.ndimage.filters import gaussian_filter

%matplotlib inline

# Download the audio data

In [5]:
def run_command(command):
    """
        Run the given command in the shell.
        input:
            command: command string to run in the shell
        output:
            returncode: return code from the shell
            ouput:      output of the command
    """
    process = subprocess.Popen(command, bufsize=2048, shell=True,
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               close_fds=(sys.platform != 'win32'))
    output = process.communicate()
    return process.returncode, output

In [9]:
video_list = ["KSW4GAx7828", "Co4FdrBZgQs", "DvmSwBBRLxw", "I6K34CYuKuE", 
              "NwIL6imI6EU", "GkOn86EtdNQ", "e9TC12UQ8og", "ajEoDMdfxFA", 
              "wA_oLYTddlA", "EM-2VXMg6VY", "zPFptdATk_s", "wFYIafs8ngw", 
              "cxwmYJ0rUvg"]
# xcZG5sIqSHE - Not downloading
save_dir = '/datadrive2/dalon/diarization-experiments/Speaker_Verification/data/youtube-audio'

for idx, videoid in enumerate(video_list):
    mp4_path = os.path.join(save_dir, videoid + '.mp4')
    wav_path = os.path.join(save_dir,videoid) + '.wav'
    print(f'Downloading ({idx + 1}/{len(video_list)}): {videoid}')
    cmd = f'youtube-dl -o "{os.path.join(save_dir,videoid)}.%(ext)s" --format mp4 https://www.youtube.com/watch?v={videoid}'
    resp, output = run_command(cmd)
    print(resp)
    if resp != 0:
        print('Failed in downloading {videoid}')
        continue
    print(f'Converting {videoid} to single channel')    
    cmd = f'ffmpeg -ac 1 -i "{mp4_path}" "{wav_path}"'
    resp, output = run_command(cmd)
    print(resp)
    if resp != 0:
        print('Failed in converting {videoid}')
        continue
    print(f'Deleting {mp4_path}')
    os.remove(mp4_path)
    print(f'Download {videoid} complete!')

0
Converting KSW4GAx7828 to single channel
0
Deleting /datadrive2/dalon/diarization-experiments/Speaker_Verification/data/youtube-audio/KSW4GAx7828.mp4
Download KSW4GAx7828 complete!
0
Converting Co4FdrBZgQs to single channel
0
Deleting /datadrive2/dalon/diarization-experiments/Speaker_Verification/data/youtube-audio/Co4FdrBZgQs.mp4
Download Co4FdrBZgQs complete!
0
Converting DvmSwBBRLxw to single channel
0
Deleting /datadrive2/dalon/diarization-experiments/Speaker_Verification/data/youtube-audio/DvmSwBBRLxw.mp4
Download DvmSwBBRLxw complete!
0
Converting I6K34CYuKuE to single channel
0
Deleting /datadrive2/dalon/diarization-experiments/Speaker_Verification/data/youtube-audio/I6K34CYuKuE.mp4
Download I6K34CYuKuE complete!
0
Converting NwIL6imI6EU to single channel
0
Deleting /datadrive2/dalon/diarization-experiments/Speaker_Verification/data/youtube-audio/NwIL6imI6EU.mp4
Download NwIL6imI6EU complete!
0
Converting GkOn86EtdNQ to single channel
0
Deleting /datadrive2/dalon/diarization-e